In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import random
import pyreadr
import time

In [ ]:
k_max = 20
k = k_max
alpha = 0.1

In [ ]:
def generate_data( n = 100000):
  X = np.random.exponential( scale=1.0, size = (n,10) )
  A = np.random.choice([0,1,2], size=n, p=[.1,.2,.7])
  Y = np.sum(X, axis = 1) + A + 5*np.random.rand(n) #######
  Y[A==1] = 10*np.random.rand(len(Y[A==1])) 
  Y = Y * np.random.rand(n) 

  df = pd.DataFrame(X, columns = ["x_"+str(_) for _ in range(10)] )
  df["a"] = pd.Series(A)
  df["y"] = pd.Series(Y)

  return df

df = generate_data( n = 100000)
xfeatures = ["x_"+str(_) for _ in range(10)]+["a"]
yfeatures = "y"
protected_features = "a"
a_unique = [0,1]

In [ ]:
## quantile regression: subject to any change of base model
def linear_quantile(X_train, y_train, X_calib, X_test, quantiles = [0.05, 0.95]):
    X_calib = X_calib.copy()
    X_test = X_test.copy()
    
    quantreg = sm.QuantReg(y_train, X_train)  # fit linear quantile model
    
    
    for q in quantiles:
        X_calib.loc[:,'qt_pred_' +str(q)] = quantreg.fit(q=q).predict(X_calib.loc[:,xfeatures])
        X_test.loc[:,'qt_pred_' +str(q)] = quantreg.fit(q=q).predict(X_test.loc[:,xfeatures])
    
    return X_calib, X_test

In [ ]:
def get_max_distance_mean(agg_l):
  a_num = len( a_unique )
  n = int(len(agg_l )/ a_num)
  bin_diff_l = []
  for i in range(n):
    max_gp, min_gp = float('-inf'), float('inf')
    gp_l = agg_l[a_num*i: a_num*(i+1)]
    for j in gp_l:
      if j>max_gp:
        max_gp = j
      if j<min_gp:
        min_gp = j
    gap = abs(max_gp - min_gp)
    gap = gap if gap <  float('inf') else 0
    bin_diff_l.append(  gap  )
  d_eop_1 = np.mean(bin_diff_l) * 100
  return d_eop_1   

In [ ]:
def get_MMD_distance(df):
  
  from sdcit.sdcit_mod import SDCIT
  from sdcit.utils import rbf_kernel_median
  
  df = df.dropna(axis=0)
  X = np.array(df["if_cover"]).reshape(-1,1)
  Y = np.array(df[protected_features]).reshape(-1,1)
  Z = np.array(df[yfeatures]).reshape(-1,1)
  Kx, Ky, Kz = rbf_kernel_median(X,Y,Z) 
  test_statistic, p_value = SDCIT(Kx, Ky, Kz)
  

In [ ]:
from more_itertools import distinct_permutations as idp
from itertools import permutations

def get_unbiased(xi,yi,yj, v1,v2):
  b1 = 1 if xi==v1 and yi==v2 else 0
  b2 = 1 if xi==v1 else 0
  b3 = 1 if yj==v2 else 0
  return b1-b2*b3

def get_4_unbiased(sample4):
  perm = list( permutations([0,1,2,3]) )
  h= 0
  for i in perm:
    for a in a_unique:
      for if_cover in [False,True]:

        #print(get_unbiased( sample4[i[0]][0], sample4[i[1]][0],sample4[i[1]][1], a, if_cover))
        h +=  get_unbiased( sample4[i[0]][0], sample4[i[1]][0], sample4[i[1]][1], a, if_cover)\
              * get_unbiased( sample4[i[2]][0], sample4[i[3]][0], sample4[i[3]][1], a, if_cover) 
  return h/ len(perm)
  

In [ ]:
# Very slow
def get_unbiased_T(test_df):
  n = len(test_df)
  poi_len = n+1
  while poi_len >= n: 
    poi_len = np.random.poisson(lam = int(n/2) )
  rows_id = random.sample( range(0,n-1), poi_len)
  selected_data = test_df.iloc[rows_id, :]

  out, bins = pd.qcut(selected_data[yfeatures], q = int(np.power(len(df), 0.4)), retbins=True , duplicates='drop')
  selected_data["y_bin"] = out
  selected_data = selected_data[[protected_features, "if_cover" ,"y_bin"]]
  selected_data.y_bin = pd.Categorical(selected_data.y_bin)
  selected_data['code'] = selected_data.y_bin.cat.codes
  selected_data = selected_data[[protected_features,"if_cover","code"]]
  
  T_list = []
  cnt = 0
  for idx,gp_df in selected_data.groupby("code"):
    gp_num = len(gp_df)
    cnt += 1
    if gp_num <=4:
      continue
    U_list = []
    total_v = 0
    estimation_data = np.array( gp_df[[protected_features,"if_cover"]] )
    estimation_data_idx = list(np.arange(len(estimation_data)))
    for permu_4 in idp( estimation_data_idx , 4):
      sample4 = estimation_data[permu_4,:]
      #print(sample4)
      res = get_4_unbiased(sample4)
      U_list.append ( res )
    U = np.mean(U_list)
    #print(cnt,"has",gp_num,"==>",U)
    T_list.append(gp_num*U)
    #print( np.sum(T_list) )
  return np.sum(T_list)

In [ ]:
from scipy.spatial.distance import cdist

def get_unbiased_V(test_df): 
  round_T = []
  for round in range(10):
      n = len(test_df)
      poi_len = n+1
      while poi_len >= n: 
        poi_len = np.random.poisson(lam = int(n/2) )
      rows_id = random.sample( range(0,n-1), poi_len)
      selected_data = test_df.iloc[rows_id, :]

      out, bins = pd.qcut(selected_data[yfeatures], q = int(np.power(len(df), 0.4)), retbins=True , duplicates='drop')
      selected_data["y_bin"] = out
      selected_data = selected_data[[protected_features, "if_cover" ,"y_bin"]]
      selected_data.y_bin = pd.Categorical(selected_data.y_bin)
      selected_data['code'] = selected_data.y_bin.cat.codes
      selected_data = selected_data[[protected_features,"if_cover","code"]]
      
      distance_list = []
      for idx,gp_df in selected_data.groupby("code"):
    
        total_v = 0
    
        n = len(gp_df)
        if n<4:
            continue
    
        one_hot_feat = pd.get_dummies( gp_df [protected_features])
        a_array, b_array = np.array( gp_df ["if_cover"]).reshape(-1,1) , np.array( one_hot_feat )
        A,B = cdist(a_array, a_array, metric='cityblock'), cdist(b_array, b_array, metric='cityblock')
        b_sum_all, b_row, b_col = np.sum(B,axis=None) , np.sum(B, axis =1 ) , np.sum(B , axis = 0 )
        B_col, B_row = np.tile(b_col,(n,1)) , np.tile(b_row,(n,1)).T
        
        total_v = ( np.sum(A*B) 
          + np.sum( A * (b_sum_all - 2*B_col - 2*B_row + 2 * B) )/ ((n-2)*(n-3)) 
          - 2 * np.sum(A * (B_row- B))/ (n-2) )/( n* (n-1) )
        distance_list.append( total_v * n )
          
      round_T.append(np.nansum(distance_list))
  return np.mean(round_T)

In [ ]:
def evaluate_fairness(eval_df):
  
  # 1. Coverage Rate of groups
  coverage_gp = eval_df[[protected_features,"if_cover"]].groupby(protected_features).mean()["if_cover"].tolist()
  d_ec = ((np.array(coverage_gp) - (1-alpha))*100).tolist()
  
  # 2 EOC Test
  # 2.1 mean max gap
  
  agg_df = eval_df.groupby(["y_bin",protected_features])['if_cover'].agg(['mean'])
  agg_l = agg_df["mean"].tolist()
  d_eop_1 = get_max_distance_mean(agg_l)
  
  # 2.2. T 
  condi_tv = get_unbiased_V(eval_df)
  #condi_tv = get_unbiased_T(eval_df)
  print("condi_tv", condi_tv)
  d_eop = (d_eop_1,condi_tv)
  
  # 3. Average Width
  interval_len = np.mean(eval_df['qt_pred_0.95'] - eval_df['qt_pred_0.05'])
  
  #print("{:.3f}%".format(d_eop))
  return d_ec,d_eop,interval_len


In [ ]:
def run_split(seed):
  global k
  
  # train: calib: test = 0.6: 0.2: 0.2
  tmp, test = train_test_split(df,train_size=0.8, shuffle=True, random_state=seed)

  # train: calib: test = 0.6: 0.2: 0.2
  train, calib = train_test_split(tmp,train_size=0.75, shuffle=True, random_state=seed)

  X_train,y_train = train.loc[:,xfeatures], train[yfeatures]

  # run qr first; do not leak bin 
  calib_qr, test_qr = linear_quantile(X_train,y_train, calib, test,[0.05, 0.95])
  #calib_qr, test_qr = nn_quantile(X_train,y_train, calib, test,[0.05, 0.95])

  # bin cali into k sets
  
  out, bins = pd.qcut(calib[yfeatures].sort_values( ascending = True), k, retbins=True , duplicates='drop')
 
  k = len(bins) - 1
  y_bin_calib = out

  # fit bin to test
  y_bin_test = pd.cut(test[yfeatures], bins=bins, include_lowest=True)

  # devide into X,y
  X_calib,y_calib = calib.loc[:,xfeatures ], calib[yfeatures]
  X_test,y_test = test.loc[:,xfeatures ], test[yfeatures]

  # evaluate orginal performace
  compare_df = test_qr
  compare_df['if_cover'] = compare_df.apply(lambda x :  (x[yfeatures] >= x['qt_pred_0.05'])&(x[yfeatures] <= x['qt_pred_0.95']), axis = 1)
  coverage_rate = compare_df['if_cover'].mean()
  compare_df["y_bin"] = y_bin_test
  d_ec,d_eop,interval_len = evaluate_fairness(compare_df)
  test_qr["y_bin"], calib_qr["y_bin"] = y_bin_test, y_bin_calib

  return coverage_rate, d_ec,d_eop, interval_len, calib_qr, X_calib,y_calib, y_bin_calib , test_qr, X_test,y_test,y_bin_test


In [ ]:
def run_vanilla_cp(y_calib,calib_qr,test_qr,y_test,X_test,y_bin_test):

  cal_labels = y_calib
  cal_upper,val_upper = calib_qr["qt_pred_0.95"],test_qr["qt_pred_0.95"]
  cal_lower,val_lower = calib_qr["qt_pred_0.05"],test_qr["qt_pred_0.05"]
  
  n = len(y_calib)

  # Get scores. cal_upper.shape[0] == cal_lower.shape[0] == n
  cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)
  # Get the score quantile
  qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')
  # Deploy (output=lower and upper adjusted quantiles)
  prediction_sets = [val_lower - qhat, val_upper + qhat]
  
  compare_df = pd.DataFrame({})
  compare_df['qt_pred_0.05'] , compare_df['qt_pred_0.95'] = prediction_sets[0], prediction_sets[1]
  compare_df[yfeatures], compare_df[protected_features] = y_test, X_test[ protected_features ]
  compare_df['if_cover'] = compare_df.apply(lambda x :  (x[yfeatures] >= x['qt_pred_0.05'])&(x[yfeatures] <= x['qt_pred_0.95']), axis = 1)
  coverage_rate = compare_df['if_cover'].mean()
  compare_df["y_bin"] = y_bin_test

  d_ec,d_eop,interval_len = evaluate_fairness(compare_df)

  return coverage_rate, d_ec,d_eop,interval_len

In [ ]:
def get_vanilla_alphas( y_bin_unique, calib_qr, y_bin_calib, y_calib):
  alpha_dict = {}
  i = 0

  cal_labels = y_calib
  cal_upper = calib_qr["qt_pred_0.95"]
  cal_lower = calib_qr["qt_pred_0.05"]
  
  n = len(y_calib)

  cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)
  qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')

  prediction_sets = [cal_lower - qhat, cal_upper + qhat]

  bin_unique = y_bin_calib.unique()
  bin_unaffected = np.zeros(k)
  #bin_none = np.zeros(k)
  for j in range(len(bin_unique)):
    bin = bin_unique[j]
    for i in range(len(y_calib)):
      res = get_intersection([bin.left, bin.right], [prediction_sets[0].iloc[i],prediction_sets[1].iloc[i] ])
      if res == None or (res[0] == bin.left) & (res[1] == bin.right):
        bin_unaffected[j] += 1

  compare_df = pd.DataFrame({})
  compare_df['qt_pred_0.05'] , compare_df['qt_pred_0.95'] = prediction_sets[0], prediction_sets[1]
  compare_df[yfeatures], compare_df[protected_features] = y_calib, calib_qr[ protected_features ]
  compare_df['if_cover'] = compare_df.apply(lambda x :  (x[yfeatures] >= x['qt_pred_0.05'])\
                  &(x[yfeatures] <= x['qt_pred_0.95']), axis = 1)
  compare_df["y_bin"] = y_bin_calib

  alpha_l =  compare_df.groupby(["y_bin"])["if_cover"].mean().tolist() 
  for i in range(k):
      alpha_dict[y_bin_unique[i]] = alpha_l[i]
  return alpha_dict,bin_unaffected

In [ ]:
def get_Q_gp_dict(y_bin_unique,df_qt_dict,protected_f_unique, alpha_dict ):
  Q_gp_dict = {}
  for y_bin in y_bin_unique:
    for protected_f in protected_f_unique:
      res = df_qt_dict[ (protected_f,y_bin) ]
      n_a_m = len(res)
      if n_a_m:
        Q_gp = np.quantile(  res , np.min( [1,np.max( [0,np.min([1,np.ceil((n_a_m + 1)*( alpha_dict[y_bin]))/n_a_m ])])]), interpolation='higher' )
        #np.quantile(  res , np.max([0,np.min([1, alpha_dict[y_bin]])]) )
        Q_gp_dict[(protected_f,y_bin)] = Q_gp
      else:
        Q_gp_dict[(protected_f,y_bin)] = 0
  return Q_gp_dict

In [ ]:
def get_intersection(interval1, interval2):
    new_min = max(interval1[0], interval2[0])
    new_max = min(interval1[1], interval2[1])
    return [new_min, new_max] if new_min <= new_max else None

def find_interval(Q_gp_dict, x, y_bin_unique):
  
  int_len, M  = 0, len(y_bin_unique)
  flag = 0
  interval_intersection_dict = {}
  first_bin, last_bin = None,None
  l,r = None, None
  for m in range(M):
    bin_flag = 0
    y_bin = y_bin_unique[m]
    Q_gp = Q_gp_dict[(x[protected_features],y_bin)]
    interval_intersection = get_intersection( [ x["qt_pred_0.05"] - Q_gp, x["qt_pred_0.95"] + Q_gp ] , [y_bin.left, y_bin.right] )
    
    if interval_intersection!= None:
      if first_bin == None:
        first_bin = y_bin
        l = interval_intersection[0] 
      last_bin = y_bin
      r = interval_intersection[1]
      int_len += interval_intersection[1] - interval_intersection[0] 
      if (x[yfeatures]<=interval_intersection[1])&(x[yfeatures]>=interval_intersection[0]):
        bin_flag, flag = 1, 1
    interval_intersection_dict[y_bin] = (interval_intersection,bin_flag)

  if r == None:
    filled_len = 0
    filled_flag = 0
  else:
    filled_len = r - l
    filled_flag = 1 if (x[yfeatures]<= r )&(x[yfeatures]>= l) else 0

  #print("int_len",int_len,"l", l ,"r",r ,"y", x[yfeatures] ,"y_lo", x["qt_pred_0.05"] ,"y_hi", x["qt_pred_0.95"] )
  return [int_len, flag, filled_len,filled_flag]

def if_bin_affected( bin, current_beta, cal_upper, cal_lower,y_calib ):

   if_lower_affected = current_beta >=  cal_lower - bin.right
   if_upper_affected = current_beta >=  bin.left - cal_upper
   return  if_upper_affected, if_lower_affected

def get_min_max_m( a, bins, Q_gp_dict, upper, lower, max = True):
  # if max == True, return the max m that has intersection
  n_bins = len(bins)
  l = range( n_bins ) if max == False else reversed( range( n_bins) )
  for m in l:
    bin = bins[m]
    Q = Q_gp_dict[(a,bin)]
    intersec = get_intersection( [bin.left,bin.right], [lower - Q, upper + Q] )
    if  intersec!= None:
      return m

In [ ]:
def get_updated_list(m, a, current_beta, Q_gp_dict, bins ,cal_upper, cal_lower,y_calib, y_bin_calib, bin_unaffected):

  bin = bins[m] 
  updated_beta = Q_gp_dict[(a,bin)] # new Q
  idx = y_bin_calib == bin
  upper, lower, y = cal_upper[idx], cal_lower[idx], y_calib[idx]
  u_cur, l_cur = if_bin_affected( bin, current_beta, upper, lower, y)
  u_new, l_new = if_bin_affected( bin, updated_beta, upper, lower, y)

  # k-: Q decreses, intersection is possible to decrease, then true->false
  if_decrease = current_beta > updated_beta
  u_changed_idx = (u_cur == if_decrease) &  ( u_new !=  if_decrease) 
  l_changed_idx = (l_cur == if_decrease) &  ( l_new !=  if_decrease) 
  u_changed_n, l_changed_n = np.sum(u_changed_idx), np.sum(l_changed_idx)

  flag = 1 if if_decrease else -1
  len_diff = 0

  bin_unaffected[m] += -flag * u_changed_n
  bin_unaffected[m] += -flag * l_changed_n

  for u_changed, l_changed in zip(upper[u_changed_idx], lower[l_changed_idx]):
    m_u = get_min_max_m( a, bins, Q_gp_dict, u_changed, l_changed, max = True)
    m_l = get_min_max_m( a, bins, Q_gp_dict, u_changed, l_changed, max = False)
    bin_unaffected[m_u] += flag
    bin_unaffected[m_l] += flag

  return bin_unaffected

In [ ]:
def opt_alpha_dict(alpha_dict,X_calib,protected_f_unique, df_qt_dict,bin_unaffected, cal_upper,cal_lower,y_calib, y_bin_calib,test_qr,test = False):

  alpha_dict_updated = alpha_dict.copy()
  #print(alpha_dict_updated)
  bins = list( alpha_dict.keys() )
  Q_gp_dict = get_Q_gp_dict(bins,df_qt_dict,protected_f_unique, alpha_dict_updated )
  log = []

  n = len(X_calib)

  k_minus_array = np.zeros(k)
  k_minus_array = np.array([np.inf]*k)
  max_step = np.zeros( (2,len(protected_f_unique), k) )

  total_step_for_run = 0

  for round in range(1000):

    # for each bin
    k_plus_bins, k_minus_bins = np.zeros(k), np.zeros(k)

    for m in range(k):
      bin = bins[m]

      # total min/max gradient of bin i
      k_minus_gp , k_plus_gp = 0,0
      for a in protected_f_unique:
       
        q_list = np.array( df_qt_dict[(a, bin)] )
        n_a_m = len(q_list)
        # update current beta
        beta = alpha_dict_updated[bin]
        
        if n_a_m:
          current_beta =  np.quantile ( q_list, np.min( [1,np.max( [0,np.min([1,np.ceil((n_a_m+1)*(beta))/n_a_m ])])]), interpolation='higher') 
        else:
          current_beta = None  
          max_step[0,a,m], max_step[1,a,m] = 1, 1
          k_minus_gp += 0
          k_plus_gp += 0
          continue

        #### minus
        try:
          if  max_step[0,a,m] <= 0:
              max_step[0,a,m] = 1/n_a_m  
          
          last_beta = np.max( q_list[ q_list< current_beta ] )
          k_minus = (current_beta-last_beta)
          k_minus_gp += k_minus * (1 - bin_unaffected[m]/n  ) 

        except:
          #print("no k_minus")
          k_minus_gp = - np.inf 

        #### plus
        try:
          if  max_step[1,a,m] <= 0:
              max_step[1,a,m] = 1/n_a_m  
 
          next_beta = np.min( q_list[ q_list> current_beta ] )
          k_plus = (next_beta-last_beta)/2
          k_plus_gp += k_plus * (1 - bin_unaffected[m]/n) 
        except:
          #print("no k_plus")
          k_plus_gp = np.inf

      k_plus_bins[m], k_minus_bins[m] = k_plus_gp, k_minus_gp #  k_plus_gp * (1 + epsilon), k_minus_gp * (1 - epsilon)   ############## test here

    # for this round, greedily rise the beta in one bin and lower the same amount of beta in another 
    k_plus_min_bin_num, k_minus_max_bin_num  = np.argmin(k_plus_bins), np.argmax( k_minus_bins )
    k_plus_min_bin,  k_minus_max_bin = k_plus_bins[k_plus_min_bin_num], k_minus_bins[k_minus_max_bin_num]

    sub_r = 0
    # note:  can not minus and plus the same bin
    while (k_plus_min_bin_num == k_minus_max_bin_num):
        k_plus_min_bin,  k_minus_max_bin = k_plus_bins[k_plus_min_bin_num], k_minus_bins[k_minus_max_bin_num]
        if sub_r > 10:
          break
        sub_r += 1
        arg_l = np.concatenate( (np.arange(0,k_plus_min_bin_num) , np.arange(k_plus_min_bin_num+1,k) ) )
        k_minus_max_bin_new = k_minus_bins[ np.argmax( k_minus_bins[arg_l] ) ]
        if k_minus_max_bin_new  > k_plus_min_bin:
          k_minus_max_bin_num = np.argmax( k_minus_bins[arg_l] )
        else:
          arg_l = np.concatenate( (np.arange(0,k_minus_max_bin_num) , np.arange(k_minus_max_bin_num+1,k) ) )
          k_plus_min_bin_num =  np.argmax( k_plus_bins[arg_l] )
          
    epsilon = 0.02
    #print("===",k_plus_min_bin,k_minus_max_bin)
    #if k_plus_min_bin * (1 + epsilon) >= k_minus_max_bin * (1-epsilon):  ############## test here
    if k_plus_min_bin + epsilon >= k_minus_max_bin: 
      break
    # find max possible step
    
    total_possible_step = np.max([ 0, np.min( max_step[:,:,k_plus_min_bin_num] )] )
    total_step_for_run += total_possible_step
    
    # update max_step 
    max_step[1,:,k_plus_min_bin_num] -= total_possible_step
    max_step[0,:,k_minus_max_bin_num] -= total_possible_step

    # update alpha dict for calibrated bin
    alpha_dict_updated[bins[k_plus_min_bin_num]] += total_possible_step
    alpha_dict_updated[bins[k_minus_max_bin_num]] -= total_possible_step

    # update affected bin
    Q_gp_dict_ori = Q_gp_dict
    Q_gp_dict = get_Q_gp_dict(bins,df_qt_dict,protected_f_unique, alpha_dict_updated )
    for a in protected_f_unique:
      for m_updated in [k_plus_min_bin_num,k_minus_max_bin_num]:
        idx = X_calib[protected_features] == a
        bin_unaffected = get_updated_list(m_updated, a, Q_gp_dict_ori[(a,bins[m_updated])], Q_gp_dict, bins ,\
                                          cal_upper[idx], cal_lower[idx], y_calib[idx], y_bin_calib[idx], bin_unaffected)
    if test and round%5==0:
      # in training epoch, see if the test interval width decreases
      X_to_use = test_qr[[ protected_features,"qt_pred_0.05",	"qt_pred_0.95", yfeatures ,"y_bin"]]
      res = X_to_use.apply(lambda x: find_interval(Q_gp_dict,x,bins), axis = 1)
      X_to_use["int_len"] = res.apply(lambda x: x[0])
      X_to_use["if_cover"] = res.apply(lambda x: x[1])
      X_to_use["filled_len"] = res.apply(lambda x: x[2])

      len_mean, cover_mean, filled_mean = np.mean(X_to_use["int_len"]), np.mean(X_to_use["if_cover"]),np.mean(X_to_use["filled_len"])
      log.append([round,len_mean, cover_mean,filled_mean])

  if not test:
    print(round, total_step_for_run)
    return alpha_dict_updated
  else:
    return log

In [ ]:
def run_my_bin_cp( X_calib, y_calib, y_bin_calib, calib_qr, X_test, y_test, test_qr, y_bin_test):

  # calculate scores of calibration data
  cal_labels = y_calib
  cal_upper,val_upper = calib_qr["qt_pred_0.95"],test_qr["qt_pred_0.95"]
  cal_lower,val_lower = calib_qr["qt_pred_0.05"],test_qr["qt_pred_0.05"]

  cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)

  y_bin_unique = y_bin_calib.unique()
  protected_f_unique = X_calib[protected_features].unique()

  # store data by group to compute quantile faster
  df_qt_dict = {}
  df_qt_comp = pd.concat([X_calib[protected_features],y_bin_calib, cal_scores],axis = 1)
  df_qt_comp.columns = [protected_features,	yfeatures, "score"]
  for a in protected_f_unique:
    for y in y_bin_unique:
      df_qt_dict[(a,y)] = df_qt_comp[(df_qt_comp[protected_features] == a) &\
                                     (df_qt_comp[yfeatures] == y)] ["score"].tolist()

  # use the proportion of groups in calib set as a initial point

  alpha_dict,bin_unaffected = get_vanilla_alphas( y_bin_unique, calib_qr, y_bin_calib, y_calib)

  # opt
  alpha_dict_updated = opt_alpha_dict(alpha_dict,X_calib,protected_f_unique,df_qt_dict,bin_unaffected, cal_upper,cal_lower,y_calib,y_bin_calib,test_qr)
  #print( "alpha_mean",np.mean( list(alpha_dict_updated.values() )) )

  # get Q for each bin according to alphas
  Q_gp_dict = get_Q_gp_dict(y_bin_unique,df_qt_dict,protected_f_unique, alpha_dict_updated)

  # apply Q to test data
  X_to_use = test_qr[[ protected_features,"qt_pred_0.05",	"qt_pred_0.95", yfeatures ,"y_bin"]]

  res = X_to_use.apply(lambda x: find_interval(Q_gp_dict,x,y_bin_unique), axis = 1)
  X_to_use["int_len"] = res.apply(lambda x: x[0])
  X_to_use["if_cover"] = res.apply(lambda x: x[1])
  #print(X_to_use.groupby(["y_bin",protected_features]).mean()["if_cover"])

  X_to_use["filled_len"] = res.apply(lambda x: x[2])
  X_to_use["filled_flag"] = res.apply(lambda x: x[3])

  # evaluate test result
  # note interval_len compuation is in different way
  d_ec,d_eop,interval_len = evaluate_fairness(X_to_use)
  interval_len =  X_to_use["int_len"].mean()
  coverage_rate = X_to_use['if_cover'].mean()
  print(coverage_rate, interval_len)

  # evaluate filled intervals
  X_filled = test_qr[[ protected_features,"qt_pred_0.05",	"qt_pred_0.95", yfeatures ,"y_bin"]]
  X_filled["int_len"], X_filled["if_cover"] = X_to_use["filled_len"] , X_to_use["filled_flag"]
  d_ec_filled,d_eop_filled,interval_len_filled = evaluate_fairness(X_filled)
  interval_len_filled = X_filled["int_len"].mean()
  coverage_rate_filled = X_filled["if_cover"].mean()

  return coverage_rate, d_ec,d_eop,interval_len,\
         coverage_rate_filled, d_ec_filled,d_eop_filled,interval_len_filled

  

In [ ]:
def run_bin_exp(seed):
  # no calibration
  c_origin, d_ec_origin, d_eop_origin, interval_len_origin, calib_qr, X_calib,y_calib, y_bin_calib , test_qr, X_test,y_test,y_bin_test = run_split(seed)
  # my bin cp
  
  c_bin, d_ec_bin, d_eopc_bin, interval_len_bin, c_filled, d_ec_filled, d_eopc_filled, interval_len_filled = \
  run_my_bin_cp( X_calib, y_calib, y_bin_calib, calib_qr, X_test, y_test, test_qr, y_bin_test  )
                                                         
  return  c_origin, d_ec_origin, d_eop_origin, interval_len_origin, c_bin, d_ec_bin, d_eopc_bin, \
           interval_len_bin, c_filled, d_ec_filled, d_eopc_filled, interval_len_filled


In [ ]:
seed = 0
c_origin, d_ec_origin, d_eop_origin, interval_len_origin, c_bin, d_ec_bin, d_eopc_bin, \
           interval_len_bin, c_filled, d_ec_filled, d_eopc_filled, interval_len_filled = run_bin_exp(seed)